In [1]:
# pip install requests beautifulsoup4 pandas

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [3]:
# Function to get HTML response
def get_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None
        
# Function to parse event URLs from a single events page
def get_all_event_urls(html):
    soup = BeautifulSoup(html, 'html.parser')
    event_links = []

    for link in soup.find_all('a', class_='b-link b-link_style_black'):
        href = link.get('href')
        if 'event-details' in href:
            event_links.append(href)
    
    return event_links

# Function to parse fight night details from an event page
def parse_fight_night_details(html):
    soup = BeautifulSoup(html, 'html.parser')
    details = {}

    details['event_name'] = soup.find('h2', class_='b-content__title').text.strip()
    details['date'] = soup.find('li', class_='b-list__box-list-item').text.strip().split(':')[-1].strip()
    details['location'] = soup.find_all('li', class_='b-list__box-list-item')[1].text.strip().split(':')[-1].strip()

    return details

In [4]:
# URL of the UFC stats completed events page
base_url = 'http://www.ufcstats.com/statistics/events/completed?page=all'

# Get all event URLs
event_urls = get_all_event_urls(get_html(base_url))

if event_urls:
    fight_nights = []
    for event_url in event_urls:
        event_html = get_html(event_url)
        if event_html:
            details = parse_fight_night_details(event_html)
            details['url'] = event_url
            fight_nights.append(details)

    # Create a DataFrame and save to a CSV file
    df = pd.DataFrame(fight_nights)
    df.to_csv('ufc_fight_nights.csv', index=False)
    print("Data has been saved to ufc_fight_nights.csv")
else:
    print("Failed to retrieve any event URLs")

Data has been saved to ufc_fight_nights.csv


In [ ]:
df